# Ibar
$\bar I = \frac{1}{2}\sum\limits_i^{species} {{m_i}z_i^2}$

In [ ]:
import sympy as sym
from sympy.printing.ccode import C99CodePrinter
sym.init_printing()

In [ ]:
nb = 5
ns = 10
ne = nb + ns
i = sym.symbols('i')

In [ ]:
b_list = []
b_T = sym.S.Zero
for j in range(0,nb):
    entry = sym.symbols('b_'+str(j))
    b_list.append(entry)
    b_T += entry
b = b_list
b, b_T

In [ ]:
T,P = sym.symbols('T P')
class s(sym.Function):
    nargs = nb+3

In [ ]:
X = s(i, T, P, *b)/b_T
X

In [ ]:
nW = b[0]
nW

In [ ]:
CapGam = sym.symbols('Gamma')
Xw = nW/b_T
Xw

In [ ]:
m = X*CapGam/Xw
m

In [ ]:
class z(sym.Function):
    nargs = 1

In [ ]:
I = m*z(i)**2/2
I

In [ ]:
class SubCodePrinter(C99CodePrinter):
    def _print_s(self, expr):
        return str(expr.func) + '(' + str(expr.args[0]) + ', ' + str(expr.args[1]) + ', ' + str(expr.args[2]) + ', b)'
    def _print_z(self, expr):
        return str(expr.func) + '(' + str(expr.args[0]) + ')'
    def _print_KroneckerDelta(self, expr):
        i = expr.args[0]
        j = expr.args[1]
        return '((i==j) ? 1 : 0)'
    def _print_Derivative(self, expr):
        function, *vars = expr.args
        number_of_derivatives = len(expr.args) - 1
        if function.func.__name__[0:1] == 's':
            if number_of_derivatives == 1:
                d_str = ''.join(repr(vars[0][0]).split('_'))
                d_ord  = '' if vars[0][1] == 1 else str(vars[0][1])
                result  = 'd' + d_ord + 's'
                for i in range(0,vars[0][1]):
                    result += 'D' + d_str
                result += '(' + str(function.args[0]) + ', ' + str(function.args[1]) + ', ' + str(function.args[2]) + ', b)'
            elif number_of_derivatives == 2:
                d_str1 = ''.join(repr(vars[0][0]).split('_'))
                d_str2 = ''.join(repr(vars[1][0]).split('_'))
                d_ord  = str(vars[0][1]+vars[1][1])
                result  = 'd' + d_ord + 's'
                for i in range(0,vars[0][1]):
                    result += 'D' + d_str1
                for i in range(0,vars[1][1]):
                    result += 'D' + d_str2
                result += '(' + str(function.args[0]) + ', ' + str(function.args[1]) + ', ' + str(function.args[2]) + ', b)'
            elif number_of_derivatives == 3:
                d_str1 = ''.join(repr(vars[0][0]).split('_'))
                d_str2 = ''.join(repr(vars[1][0]).split('_'))
                d_str3 = ''.join(repr(vars[2][0]).split('_'))
                d_ord  = str(vars[0][1]+vars[1][1]+vars[2][1])
                result  = 'd' + d_ord + 's'
                for i in range(0,vars[0][1]):
                    result += 'D' + d_str1
                for i in range(0,vars[1][1]):
                    result += 'D' + d_str2
                for i in range(0,vars[2][1]):
                    result += 'D' + d_str3
                result += '(' + str(function.args[0]) + ', ' + str(function.args[1]) + ', ' + str(function.args[2]) + ', b)'
            else:
                print ('Error: More than three derivatives of s not supported.')
                result = ''
        else:
            if (not isinstance(type(function), UndefinedFunction) or 
                not all(isinstance(i, Symbol) for i in vars)):
                return super()._print_Derivative(expr)
        return result
printer = SubCodePrinter()

In [ ]:
ibar_template = \
"""
double Ibar = 0.0;
for (int i=0; i<ne; i++) {{
    Ibar = {ibar};
}}
"""
print (ibar_template.format(ibar=printer.doprint(I).replace('b_0', 'b[0]')))

In [ ]:
I.diff(b[0])

In [ ]:
header = \
"""
double dIbarDb[nb];
for (int i=0; i<ne; i++) {"""
footer = \
"""}
"""
print (header)
for i in range(0,nb):
    print ('   ', printer.doprint(I.diff(b[i]), assign_to='dIbarDb['+str(i)+']').replace('b_0', 'b[0]'))
print (footer)

In [ ]:
I.diff(b[0],b[1])

In [ ]:
header = \
"""
double d2IbarDb2[nb][nb];
for (int i=0; i<ne; i++) {"""
footer = \
"""}
"""
print (header)
for i in range(0,nb):
    for j in range(i,nb):
        print ('   ', printer.doprint(I.diff(b[i], b[j]), assign_to='d2IbarDb2['+str(i)+']['+str(j)+']').replace('b_0', 'b[0]'))
        if j > i:
            print ('   ', 'd2IbarDb2['+str(j)+']['+str(i)+'] = d2IbarDb2['+str(i)+']['+str(j)+']')
print (footer)

In [ ]:
I.diff(b[0],b[1],b[2])

In [ ]:
header = \
"""
double d3IbarDb3[nb][nb][nb];
for (int i=0; i<ne; i++) {"""
footer = \
"""}
"""
print (header)
for i in range(0,nb):
    for j in range(i,nb):
        for k in range(j,nb):
            print('   ', printer.doprint(I.diff(b[i],b[j],b[k]), 
                                         assign_to='d3IbarDb3['+str(i)+']['+str(j)+']['+str(k)+']').replace('b_0', 'b[0]'))
            if k > j:
                print('   ', 'd3IbarDb3['+str(i)+']['+str(k)+']['+str(j)+'] = d3IbarDb3['+str(i)+']['+str(j)+']['+str(k)+']')
        if j > i:
            for k in range(i, nb):
                print('   ', 'd3IbarDb3['+str(j)+']['+str(i)+']['+str(k)+'] = d3IbarDb3['+str(i)+']['+str(j)+']['+str(k)+']')
                if k > i:
                    print('   ', 'd3IbarDb3['+str(j)+']['+str(k)+']['+str(i)+'] = d3IbarDb3['+str(i)+']['+str(j)+']['+str(k)+']')
print (footer)